# **Q1**

In [14]:
#importing .env variables to use openai api key
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
DEFAULT_MODEL = "gpt-4o-mini"

In [15]:
import json
import os
from dotenv import load_dotenv
import openai

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_model_response(conversation: str, prompt: str, model: str = DEFAULT_MODEL) -> dict:
    """
    Get response from OpenAI model with conversation and prompt
    
    Args:
        conversation (str): The customer-agent conversation
        prompt (str): The system prompt for the task
        model (str): OpenAI model to use
    
    Returns:
        dict: Parsed JSON response from the model
    """
    try:
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": conversation}
            ],
            temperature=0.1,  # Lower temperature for more consistent results
            response_format={"type": "json_object"}  # Ensures JSON response
        )
        
        # Parse the JSON response
        result = json.loads(response.choices[0].message.content)
        return result
        
    except json.JSONDecodeError as e:
        return {"error": f"Failed to parse JSON: {str(e)}"}
    except Exception as e:
        return {"error": f"API call failed: {str(e)}"}

# Prompt for Task 1: Message Classification


In [16]:
""""
While writing a prompt for a language model we need to follow 5-6 important steps: 
1. context : Provide context to the model about the task at hand.
2. persona : Define the role or persona the model should assume.
3. Examples : Provide examples to guide the model's responses.
4. task : Clearly state the task you want the model to perform.
5. format : Specify the desired format of the output.
6. Tone(optional) : Define the tone of the response if necessary like "respond in a professional tone" or "use casual language".
"""
# Prompt for Task 1: Message Classification

prompt_1 = """
[CONTEXT]
You are analyzing a customer service conversation between a customer and a company representative.
The entire conversation will be given to you as a single plain-text string. Messages are usually written one after another and may start with labels like "Customer:" or "Agent:".

[PERSONA]
Act as a professional conversation analyst for a contact center.
Be precise and consistent in your classifications. If you are uncertain, choose the most reasonable category and use a lower confidence value.

[EXAMPLES]
Interpret the conversation and identify what each customer message is doing:
- Asking for help with a problem (support_request)
- Asking about products, prices, upgrades, or purchases (sales_inquiry)
- Expressing dissatisfaction or annoyance (complaint)
- Praising the company or service (compliment)

For example:
- "I can't log in to my account" → support_request
- "How much does your premium plan cost?" → sales_inquiry
- "It's really annoying that this keeps happening" → complaint
- "Your service is always excellent" → compliment

[TASK]
1. Read the entire conversation string.
2. Break it into individual messages (for example by line breaks and/or labels like "Customer:" / "Agent:").
3. Consider only **customer** messages for classification.
4. For each customer message that clearly fits one of the categories, create one classification item:
   - support_request: customer needs help or reports a problem.
   - sales_inquiry: customer asks about plans, pricing, products, or upgrades.
   - complaint: customer expresses dissatisfaction, annoyance, or frustration.
   - compliment: customer expresses satisfaction or praise.
5. If a customer message does not clearly match any category, do not include it.
6. If needed, you may extract just the relevant part of a longer message as the "text_excerpt".

[FORMAT]
Return a single valid JSON object (no extra text) with this structure:

{
  "classifications": [
    {
      "message_number": <integer, 1-based index of the message in the conversation, counting all messages>,
      "speaker": "customer",
      "text_excerpt": "<short relevant part of the customer's message>",
      "category": "support_request" | "sales_inquiry" | "complaint | "compliment",
      "confidence": <float between 0.0 and 1.0>
    }
  ]
}

Requirements:
- "classifications" must always exist (it can be an empty list).
- Only include items where speaker is "customer".
- Ensure the JSON is syntactically valid and parseable.
- Respond with JSON only, no explanations or extra text.
"""

In [17]:
conversation_1 = """
Customer: Hello, I have a problem with my account. Suddenly I can't log in anymore. 
Agent: Hi, I'd be happy to help! Are you getting any specific error message?
Customer: Yes, it says the password is incorrect, but I didn't change it. 
Agent: I understand, let's try resetting the password. 
Customer: Wait, before that – I'm also considering upgrading my plan. How much does the extended plan cost?
Agent: The extended plan costs 79 NIS per month.
Customer: Thanks. By the way, your service is always excellent, really.
Agent: Thank you very much, it's great to hear!
Customer: But honestly it's pretty annoying that the system is locked for me again. It happens about once a month.
"""

In [18]:
get_model_response(conversation_1, prompt_1)

{'classifications': [{'message_number': 1,
   'speaker': 'customer',
   'text_excerpt': "I have a problem with my account. Suddenly I can't log in anymore.",
   'category': 'support_request',
   'confidence': 1.0},
  {'message_number': 3,
   'speaker': 'customer',
   'text_excerpt': "I'm also considering upgrading my plan. How much does the extended plan cost?",
   'category': 'sales_inquiry',
   'confidence': 1.0},
  {'message_number': 5,
   'speaker': 'customer',
   'text_excerpt': 'your service is always excellent, really.',
   'category': 'compliment',
   'confidence': 1.0},
  {'message_number': 7,
   'speaker': 'customer',
   'text_excerpt': "it's pretty annoying that the system is locked for me again.",
   'category': 'complaint',
   'confidence': 1.0}]}

# Q2

In [19]:
TV_ANALYSIS_PROMPT = """
[CONTEXT]
You are analyzing a customer service conversation between a customer and a representative of a television company.
The entire conversation will be provided as a single plain-text string.

[PERSONA]
Act as a business analyst focused on customer experience and product improvement.
You are concise, practical, and focus on what is most useful for product, support, and management teams.

[EXAMPLES]
Interpret the conversation and extract:
- Overall customer satisfaction level (high / medium / low)
- The main issues or pain points mentioned by the customer
- Concrete suggestions or improvements requested by the customer, or clearly implied by their issues

For example:
- If the customer is mostly positive, with minor issues that get resolved → satisfaction is "high".
- If the customer mentions repeated problems or serious frustration → satisfaction is "low".
- If the customer asks for better onboarding or clearer setup instructions → an improvement suggestion could be "add clearer setup instructions or a quick-start guide".

[TASK]
Given the full conversation as a single string:
1. Read the entire conversation and understand the customer's experience from beginning to end.
2. Determine the overall customer_satisfaction: "high", "medium", or "low", based on both explicit feedback and emotional tone.
3. Identify the main_issues:
   - Summarize the key problems, difficulties, or friction points the customer mentions.
   - Use short, clear descriptions (e.g., "difficulty connecting to WiFi", "confusing initial setup").
4. Identify improvement_suggestions:
   - Extract any explicit suggestions the customer makes (e.g., "add a short tutorial video").
   - Add any obvious, actionable improvements implied by the issues (e.g., "simplify WiFi setup steps").

[FORMAT]
Return a single valid JSON object (no extra text) with exactly these 3 keys:

{
  "customer_satisfaction": "high" | "medium" | "low",
  "main_issues": ["..."],
  "improvement_suggestions": ["..."]
}

Requirements:
- "main_issues" is a JSON array of short strings; if there are no clear issues, return an empty array.
- "improvement_suggestions" is a JSON array of short strings; if there are no clear suggestions, return an empty array.
- Do not add any keys other than the three specified.
- Respond only with the JSON object, no explanations or surrounding text.
"""

In [20]:

conversation_2 = """
Customer: Hey, I bought your new device and I received it yesterday.
Agent: Hey! Great, how did the initial setup feel?
Customer: At first it was a bit complicated, but after I read the manual it started working.
Agent: Good to hear! Did you have any specific problems or is everything okay?
Customer: Yes, there was a problem with the WiFi connection, it took me a few tries.
Agent: Understood, that happens sometimes. Were you able to connect in the end? 
Customer: Yes, now everything is fine. But I'd be happy if in the future you could add a short tutorial video; it helps a lot.
Agent: Thanks for the suggestion, I'll definitely pass it on to the development team.
Customer: Thank you very much, your service is excellent, it makes a good impression.
"""

In [21]:
get_model_response(conversation_2, TV_ANALYSIS_PROMPT)

{'customer_satisfaction': 'medium',
 'main_issues': ['complicated initial setup', 'difficulty connecting to WiFi'],
 'improvement_suggestions': ['add a short tutorial video']}